In [2]:
# 📌 PC1~PC5에서 반복적으로 중요한 변수:
pca_cols = [
    'CA이자율_할인전', 'CL이자율_할인전', 'RV_평균잔액_R3M', 'RV일시불이자율_할인전', 'RV최소결제비율', 'RV현금서비스이자율_할인전',
    '방문월수_앱_R6M', '방문일수_앱_B0M', '방문일수_앱_R6M', '방문횟수_앱_B0M', '방문후경과월_앱_R6M',
    '이용금액_R3M_신용', '이용금액_R3M_신용체크', '이용금액_일시불_B0M', '이용금액대',
    '일시불ONLY전환가능여부',
    '잔액_리볼빙일시불이월_B0M', '잔액_일시불_B0M', '잔액_일시불_B1M', '잔액_일시불_B2M', '잔액_카드론_B0M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M',
    '정상청구원금_B0M', '정상청구원금_B2M', '정상청구원금_B5M',
    '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '최종카드론_대출금액', '카드론이용금액_누적', '평잔_RV일시불_3M', '평잔_RV일시불_6M', '평잔_일시불_3M', '평잔_일시불_6M',
    '평잔_카드론_3M', '평잔_카드론_6M', '평잔_할부_3M', '홈페이지_금융건수_R3M', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R3M', '홈페이지_선결제건수_R6M'
    ]

In [3]:
pca_cols = list(set(pca_cols))  # 또는 수동으로 중복 제거

In [4]:
# Recency
recency_cols = [
    '입회경과개월수_신용'
]

# Frequency (최근 6개월간 이용건수)
frequency_cols = [
    '이용건수_신용_R6M',
    '이용건수_체크_R6M'
]

# Monetary (최근 6개월간 총 이용금액)
monetary_cols = [
    '이용금액_일시불_R6M',
    '이용금액_할부_R6M',
    '이용금액_CA_R6M',
    '이용금액_체크_R6M',
    '이용금액_카드론_R6M'
]

# 최종 컬럼 목록
rfm_cols = recency_cols + frequency_cols + monetary_cols

In [5]:
base_cols = ["ID","Segment"]
selected_cols = pca_cols + base_cols + rfm_cols

In [5]:
import pandas as pd

# parquet에서 필요한 컬럼만 불러오기
file_path = "data/통합_train_데이터.parquet"
df = pd.read_parquet(file_path, columns=selected_cols)

In [6]:
# Recency는 보통 단일 컬럼이라면 .squeeze() 또는 [0] 인덱스로 접근
df['Recency'] = df[recency_cols[0]]

# Frequency와 Monetary는 여러 컬럼을 합산하는 것이라 문제 없음
df['Frequency'] = df[frequency_cols].sum(axis=1)
df['Monetary'] = df[monetary_cols].sum(axis=1)

In [7]:
df = df.drop(columns=rfm_cols)

In [8]:
# 1. Recency: 오래될수록 좋은 고객 -> 높을수록 점수 높게

df['Recency'] = df['Recency'].apply(lambda x: max(x, 0))
df['R_score'] = pd.qcut(df['Recency'], 4, labels=[1,2,3,4])


# 2. Frequency: 높을수록 점수 높게

df['Frequency'] = df['Frequency'].apply(lambda x: max(x, 0))
df['F_score'] = pd.qcut(df['Frequency'], 4, labels=[1,2,3,4])

# 3. Monetary: 높을수록 점수 높게

df['Monetary'] = df['Monetary'].apply(lambda x: max(x, 0))
df['M_score'] = pd.qcut(df['Monetary'], 4, labels=[1,2,3,4])

### 트레인데이터

In [9]:
print(df['R_score'].value_counts().sort_index())
print(df['F_score'].value_counts().sort_index())
print(df['M_score'].value_counts().sort_index())

R_score
1    617474
2    588708
3    599235
4    594583
Name: count, dtype: int64
F_score
1    607574
2    593647
3    599365
4    599414
Name: count, dtype: int64
M_score
1    600041
2    599981
3    599980
4    599998
Name: count, dtype: int64


In [10]:
# pca + rfm
baseline_with_rfm = df.drop(columns = ['Recency', 'Frequency', 'Monetary'])

# rfm만
rfm_only = df[['R_score', 'F_score', 'M_score', 'ID', 'Segment']]

In [11]:
# # Parquet 저장
# rfm_only.to_parquet("data/rfm_only.parquet", index=False, encoding='utf-8-sig')

# 또는 CSV로 저장하고 싶다면
rfm_only.to_csv("data/rfm_only.csv", index=False, encoding='utf-8-sig')

In [12]:
10/0

ZeroDivisionError: division by zero

### 테스트 데이터

In [6]:
base_cols = ["ID"]
selected_cols = pca_cols + base_cols + rfm_cols

In [7]:
import pandas as pd

# parquet에서 필요한 컬럼만 불러오기
file_path = "data/통합_test_데이터.parquet"
df = pd.read_parquet(file_path, columns=selected_cols)

In [8]:
# Recency는 보통 단일 컬럼이라면 .squeeze() 또는 [0] 인덱스로 접근
df['Recency'] = df[recency_cols[0]]

# Frequency와 Monetary는 여러 컬럼을 합산하는 것이라 문제 없음
df['Frequency'] = df[frequency_cols].sum(axis=1)
df['Monetary'] = df[monetary_cols].sum(axis=1)

In [9]:
df = df.drop(columns=rfm_cols)

In [10]:
# 1. Recency: 오래될수록 좋은 고객 -> 높을수록 점수 높게

df['Recency'] = df['Recency'].apply(lambda x: max(x, 0))
df['R_score'] = pd.qcut(df['Recency'], 4, labels=[1,2,3,4])


# 2. Frequency: 높을수록 점수 높게

df['Frequency'] = df['Frequency'].apply(lambda x: max(x, 0))
df['F_score'] = pd.qcut(df['Frequency'], 4, labels=[1,2,3,4])

# 3. Monetary: 높을수록 점수 높게

df['Monetary'] = df['Monetary'].apply(lambda x: max(x, 0))
df['M_score'] = pd.qcut(df['Monetary'], 4, labels=[1,2,3,4])

In [11]:
print(df['R_score'].value_counts().sort_index())
print(df['F_score'].value_counts().sort_index())
print(df['M_score'].value_counts().sort_index())

R_score
1    154635
2    147670
3    148389
4    149306
Name: count, dtype: int64
F_score
1    153280
2    148589
3    148991
4    149140
Name: count, dtype: int64
M_score
1    150001
2    149999
3    150000
4    150000
Name: count, dtype: int64


In [12]:
# pca + rfm
baseline_with_rfm = df.drop(columns = ['Recency', 'Frequency', 'Monetary'])

In [15]:
# RFM 점수만 추출
rfm_test_only = df[['R_score', 'F_score', 'M_score', 'ID']]

# # Parquet 저장
# rfm_test_only.to_parquet("data/rfm_test_only.parquet", index=False)

# 또는 CSV 저장
rfm_test_only.to_csv("data/rfm_test_only.csv", index=False, encoding='utf-8-sig')